In [2]:
import cv2
import numpy as np
import os
from skimage import feature
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score


from sklearn.cluster import KMeans
import pickle
from scipy.spatial.distance import cdist

# Path to the folder containing all the images
path = "/Users/michelle/Desktop/W281/final_project/rendered_256x256/256x256/sketch/tx_000100000000"
#SKETCH_ROOT = "/home/sysung98/MIDS/W281/final_project/data/rendered_256x256/256x256/sketch"
#path = SKETCH_ROOT + '/tx_000100000000'

# List all the subdirectories (i.e., categories) in the main folder
categories = os.listdir(path)
categories = [x for x in categories if x != '.DS_Store']
categories = ['apple']
len(categories)

1

In [3]:
def edge_detection(sketch_img_fp):
    ''' 
    apply canny edge detector to image 

    returns
        np.argwhere(canny_img==1)       An array of edges (1-D)
        canny_img.astype(int) * 256     Image of canny (2-D 256x256)
    '''
    sketch_img = cv2.imread(sketch_img_fp)
    gray = cv2.cvtColor(sketch_img, cv2.COLOR_BGR2GRAY)

    # canny_img = skimage.feature.canny(
    #     gray, 
    #     sigma=4,
    #     high_threshold=60
    # )

    canny_img = feature.canny(
        gray
    )

    return np.argwhere(canny_img==1), canny_img.astype(int) * 256

In [4]:
%%time
# Define a list to store the descriptors of all the images
descriptors = []

# Define a list to store the corresponding labels of all the images
labels = []

# Loop through all the categories
for i, category in enumerate(categories):
    # Get the path to the category folder
    category_path = os.path.join(path, category)
    # List all the images in the category folder
    images = os.listdir(category_path)
    count = 1
    
    # Loop through all the images in the category folder
    for image in images:
        if count <= 1:
            # Get the path to the image
            image_path = os.path.join(category_path, image)
            # Apply edge detector
            _, edge_img = edge_detection(image_path)
            # Store the descriptors in the descriptors list
            descriptors.append(edge_img.flatten())
            # Store the label i
            labels.append(i)

CPU times: user 6.63 s, sys: 679 ms, total: 7.31 s
Wall time: 7.54 s


In [9]:
len(descriptors[0])

65536

In [4]:
#max_len = max([len(x) for x in descriptors])
#new_arr = [np.pad(x, (0, (max_len-len(x)))) for x in descriptors]
X_train, X_test, Y_train, Y_test = train_test_split(descriptors, labels, test_size = 0.2, random_state = 1)

In [1]:
%%time
# Train an SVM with linear kernel
svm_model = SVC(C=30, kernel='linear', random_state=0)
svm_model.fit(X_train, Y_train)
    
# Evaluate the performance of SVM model
svm_preds = svm_model.predict(X_test)
svm_acc = accuracy_score(Y_test, svm_preds)
print(f'SVM accuracy: {svm_acc}')

NameError: name 'SVC' is not defined